# PREPARE DATA

The *main_figures.ypynb* and *supp_figures.ipynb* notebooks generate all plots in the manuscript, but in order to do so they make use of supplementary files that are generated by running this notebook (this may take some time). 

In [2]:
from functions import *

# PREPARE DATASET
First we need to download the ATLAS dataset (https://amr.theodi.org/programmes/atlas) from the following URL:
https://s3-eu-west-1.amazonaws.com/amr-prototype-data/Open+Atlas_Reuse_Data.xlsx

Then run the next cell.

In [ ]:
#WE READ THE EXCEL FILE
DF=pd.read_excel('data/Open+Atlas_Reuse_Data.xlsx', skiprows=[0,1,2,3,4])
DF=DF.rename(columns={'In / Out Patient': 'InOutPatient'})
#WRITE DATAFRAME'S KEYS (USEFUL FOR LATER ANALYSES)
keys=['Year', 'Country', 'Species']
mkeys={}
for key in keys:
    dL=DF[key].tolist()
    mkeys[key]={}
    for i,z in enumerate(dL):
        z=str(z)
        if z not in mkeys[key]:
            mkeys[key][z]=0  
        mkeys[key][z]+=1
    with open('data/key_'+key+'.txt','w') as f:
        for z in sorted(mkeys[key], key=lambda x: (str(type(x)), x)):
            f.write(str(z)+'\t'+str(mkeys[key][z])+'\n')

#WE WANT TO LOG2-TRANSFORM THE MIC VALUES IN THE DATASET
#BUT MIC VALUES APPEAR IN A VARIETY OF WAYS THROUGHOUT THE DATASET
#FILE data/key_MIC_values.txt HOLDS ALL MIC VALUES IN THE DATASET
#IN THE FIRST COLUMN. IN THE SECOND WE HAVE MODIFIED THEM, AND IN THE
#THIRD WE HAVE THE LOG2-TRANSFORMED VALUES
MICdict=[]
with open('data/key_MIC_values.txt', 'r') as f:
    for line in f:
        w=line.split()
        k1=w[0]
        k3=float(w[2])
        MICdict.append([k1, str(k3), len(k1)])
MICdict=sorted(MICdict, key=itemgetter(2), reverse=True)

#WE ASSIGN A RANDOM KEY TO EACH MIC VALUE BEFORE SUBSTITUTING
import string
import random
def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))
eggs=[randomString() for z in MICdict]

#NOW WE SUBSTITUTE THEM IN THE DATASET
drugs=read_key('Drugs', 'data/')
for dr in drugs:
    DF[dr]=DF[dr].astype('str')
    for x1,y1 in zip(MICdict,eggs):
        DF[dr]=DF[dr].str.replace(x1[0],y1)
    for x1,y1 in zip(eggs,MICdict):
        DF[dr]=DF[dr].str.replace(x1,y1[1])
    DF[dr]=DF[dr].astype('float64')
DF['Species']=DF['Species'].astype('str')
#FINALLY, WE UNIFORMIZE SOME SPECIES
DF['Species']=DF['Species'].str.replace('Enterobacter aerogenes', 'Klebsiella aerogenes', regex=False)
DF['Species']=DF['Species'].str.replace('Klebsiella (Enterobacter) aerogenes', 'Klebsiella aerogenes', regex=False)
DF['Species']=DF['Species'].str.replace('Klebsiella planticola', 'Raoultella planticola', regex=False)
DF.to_excel('data/atlas_treated.xlsx')   
#WRITE NEW SPECIES FILE
mkeys={}
dL=DF['Species'].tolist()
for i,z in enumerate(dL):
    z=str(z)
    if z not in mkeys:
        mkeys[z]=0  
    mkeys[z]+=1
with open('data/key_Species.txt','w') as f:
    for z in sorted(mkeys, key=lambda x: (str(type(x)), x)):
        f.write(str(z)+'\t'+str(mkeys[z])+'\n')

# DATAPOINTS PER YEAR
Datafile to represent how many species and datapoints are recorded each year in ATLAS (Fig S1D).

In [ ]:
DF=pd.read_excel('data/atlas_treated.xlsx')
yL=list(range(2004,2018))
spL=[]
dataL=[]
for year in yL:
    DFy=DF[DF.Year==year]
    k1=get_key(DFy,'Species')
    dataL.append(len(DFy))
    spL.append(len(k1))
DFnew=pd.DataFrame({'Year':yL, 'Species': spL, 'Datapoints': dataL})
DFnew.to_csv('data/datapoints_per_year.csv', index=False)

# READ DATASET
Useful for later cells.

In [ ]:
DF, pL, bP, drugs=read_dataset('data/')

# AVERAGE MIC PER YEAR
Datafile to represent average MIC for each country (Figs 2A, 2C).

In [ ]:
L=[]
pC=get_key(DF,'Country')
for country in pC:#for every country
    DFc=DF[DF.Country==country]
    if DFc.empty: continue   
    for sp in pL:#for every pathogen
        DFsp=DFc[DFc.Species==sp]
        if DFsp.empty: continue
        for dr in drugs:#for every drug
            if dr not in bP[sp]: continue
            DFnew=pd.DataFrame({'Year': DFsp['Year'], dr: DFsp[dr]})
            DFnew=DFnew.dropna()
            if DFnew.empty: continue
            yL=get_key(DFnew, 'Year')
            for y in yL:#for every year
                DFy=DFnew[DFnew.Year==y]
                if DFy.empty: continue
                #DATA
                rcases=DFy[DFy[dr]>bP[sp][dr][0]].count()[dr]  
                m=DFy[dr].mean()-bP[sp][dr][0]
                merr=DFy[dr].sem()
                mstd=DFy[dr].std()
                N=len(DFy)
                R=rcases
                F=rcases/N
                #RECORD
                L.append([sp,dr,country,y,m,merr,mstd,N,R,F])
#SAVE ALL IN DATAFRAME AND WRITE TO FILE
MICdf=pd.DataFrame.from_records(L, columns=['Species', 'Antibiotic', 'Country', 'Year', 'Average MIC', 'Average MIC SEM', 'Average MIC SD', 'Number of cases', 'Number of resistant cases', 'Fraction of resistant cases'])
MICdf.to_csv('data/average_mic_per_year.csv', float_format='%.3f', na_rep='NaN', index=False)

# COMPARE DATASETS
Compare resistance values in ATLAS against ECDC and ResistanceMap (Figs 1A,1B,S2).

## ATLAS VS ECDC

In [5]:
#READ AVERAGE MIC DATA
DFavg=pd.read_csv('data/average_mic_per_year.csv')
#READ ECDC DATA
RD=pd.read_csv('data/europe_resistance_data.csv')
#we want to compare fraction of resistance in both datasets
RD=RD[RD.Indicator=='Nonsusceptible proportion']
RD['Year']=RD['Year'].astype('int64')
RD['Value']=RD['Value'].astype('float64')
spR=get_key(RD, 'Species')
cR=get_key(RD, 'Country')
dR=get_key(RD, 'Antibiotic')
#ECDC resistance data comes aggregated for drug classes and for different
#pathogens. For instance, for Acinetobacter it reports resistance to
#'Carbapenems'. We therefore compare with resistance to all drugs tested
#in ATLAS for that pathogen-drug class combination, as specified in the
#ECDC website.
#Also, Acinetobacter is reported as a genus, but we here only compare with
#A.baumannii, which is by far the most important pathogen
RD['Species']=RD['Species'].str.replace('Acinetobacter spp.', 'Acinetobacter baumannii')
Rant={}
Rant['Acinetobacter baumannii.']={'Carbapenems':['Imipenem', 'Meropenem']}
Rant['Enterococcus faecalis']={'Highlevel gentamicin':['Gentamicin'],
    'Vancomycin':['Vancomycin']}
Rant['Enterococcus faecium']={'Highlevel gentamicin':['Gentamicin'],
    'Vancomycin':['Vancomycin']}
Rant['Escherichia coli']={'Carbapenems':['Imipenem', 'Meropenem']}
Rant['Klebsiella pneumoniae']={'Carbapenems':['Imipenem, Meropenem']}
Rant['Pseudomonas aeruginosa']={'Carbapenems':['Imipenem', 'Meropenem'],
    'Ceftazidime':['Ceftazidime'],
    'PiperacillinTazobactam':['Piperacillin tazobactam']}
Rant['Staphylococcus aureus']={'Meticillin (MRSA)':['Oxacillin']}
Rant['Streptococcus pneumoniae']={'Macrolides':['Erythromycin', 'Clarithromycin', 'Azithromycin'],
    'Penicillins': ['Penicillin', 'Oxacillin']}

#COMPARE RESISTANCE RATES
yL=list(range(2004,2018))
L=[]
for sp, country in itertools.product(spR, cR):
    DFsp=DFavg[(DFavg.Species==sp) & (DFavg.Country==country)]
    RDsp=RD[(RD.Species==sp) & (RD.Country==country)]
    if DFsp.empty or RDsp.empty: continue
    for key,value in Rant[sp].items():
        DFc=DFsp[DFsp.Antibiotic.isin(value)]
        RDc=RDsp[RDsp.Antibiotic==key]
        for y in yL:
            DFy=DFc[DFc.Year==y]
            RDy=RDc[RDc.Year==y]
            if DFy.empty or RDy.empty: continue
            x1=RDy.Value.mean()
            if np.isnan(x1): continue
            y1=DFy['Fraction of resistant cases'].mean()*100
            diff=y1-x1
            N=DFy['Number of cases'].sum()
            L.append([sp,key,y,country,x1,y1,diff,N])
NewDF=pd.DataFrame.from_records(L, columns=['Species','Antibiotic','Year','Country','ECDCValue','ATLASValue','Diff','N'])
NewDF.to_excel('data/atlas_vs_ecdc.xls', float_format='%.3f', index=False)

## ATLAS VS RESISTANCEMAP

In [7]:
#READ AVERAGE MIC DATA
DFavg=pd.read_csv('data/average_mic_per_year.csv')
#READ RESISTANCEMAP DATA
RD=pd.read_csv('data/resistmap_data.csv')
RD['year']=RD['year'].astype('int64')
RD['value']=RD['value'].astype('float64')
RD['IsolatesTested']=RD['IsolatesTested'].astype('float64')
spR=get_key(RD, 'Organism')
cR=get_key(RD, 'CountryName')
dR=get_key(RD, 'Antimicrobial')
#DATA IN RESISTANCEMAP IS AGGREGATED IN A SIMILAR WAY AS IN ECDC
Rant={}
Rant['Acinetobacter baumannii']={'Amikacin':['Amikacin'], 
    'Ampicillin-sulbactam':['Ampicillin sulbactam'],
    'Carbapenems':['Imipenem', 'Meropenem'],
    'Ceftazidime':['Ceftazidime'],
    'Glycylcyclines':['Tigecycline']}
Rant['Enterobacter aerogenes/cloacae']={'Amoxicillin-clavulanate':['Amoxicillin clavulanate'],
    'Carbapenems':['Imipenem', 'Meropenem'],
    'Glycylcyclines':['Tigecycline'],
    'Piperacillin-tazobactam':['Piperacillin tazobactam']}
Rant['Enterococcus faecalis']={'Aminoglycosides (high-level)':['Gentamicin'],
    'Vancomycin':['Vancomycin']}
Rant['Enterococcus faecium']={'Vancomycin':['Vancomycin']}
Rant['Escherichia coli']={'Amoxicillin-clavulanate':['Amoxicillin clavulanate'],
    'Carbapenems':['Imipenem', 'Meropenem'],
    'Glycylcyclines':['Tigecycline'],
    'Piperacillin-tazobactam':['Piperacillin tazobactam']}
Rant['Klebsiella pneumoniae']={'Amoxicillin-clavulanate':['Amoxicillin clavulanate'],
    'Carbapenems':['Imipenem, Meropenem'],
    'Glycylcyclines':['Tigecycline'],
    'Piperacillin-tazobactam':['Piperacillin tazobactam']}
Rant['Pseudomonas aeruginosa']={'Amikacin':['Amikacin'],
    'Carbapenems':['Imipenem', 'Meropenem'],
    'Ceftazidime':['Ceftazidime'],
    'Piperacillin-tazobactam':['Piperacillin tazobactam']}
Rant['Staphylococcus aureus']={'Linezolid':['Linezolid'],
    'Oxacillin (MRSA)':['Oxacillin'],
    'Vancomycin':['Vancomycin']}
Rant['Streptococcus pneumoniae']={'Macrolides':['Erythromycin', 'Clarithromycin', 'Azithromycin']}

#COMPARE RESISTANCE RATES
yL=list(range(2004,2018))
L=[]
for sp, country in itertools.product(spR, cR):
    DFsp=DFavg[(DFavg.Species==sp) & (DFavg.Country==country)]
    RDsp=RD[(RD.Organism==sp) & (RD.CountryName==country)]
    if DFsp.empty or RDsp.empty: continue
    for key,value in Rant[sp].items():
        DFc=DFsp[DFsp.Antibiotic.isin(value)]
        RDc=RDsp[RDsp.Antimicrobial==key]
        for y in yL:
            DFy=DFc[DFc.Year==y]
            RDy=RDc[RDc.year==y]
            if DFy.empty or RDy.empty: continue
            x1=RDy.value.mean()
            xhigh=RDy.CIHigh.mean()
            xlow=RDy.CILow.mean()
            if np.isnan(x1): continue
            y1=DFy['Fraction of resistant cases'].mean()*100
            diff=y1-x1
            N=DFy['Number of cases'].sum()
            L.append([sp,key,y,country,x1,xhigh,xlow,y1,diff,N])

NewDF=pd.DataFrame.from_records(L, columns=['Species','Antibiotic','Year','Country','RMapValue','RMapCIHigh', 'RMapCILow','ATLASValue','Diff','N'])
NewDF.to_excel('data/atlas_vs_resistancemap.xls', float_format='%.3f', index=False)

# ATLAS QUALITY
Datafiles to represent correlelograms (Figs 1C, 1D, S3A, S3B, S4, S5, S12)

In [ ]:
#CREATE NEW DIRECTORY
import os
os.mkdir('results/')
os.mkdir('results/correlations')

#COMPUTE CORRELATIONS BETWEEN YEARS FOR EACH PA
bins=np.arange(-10.5,11.5,1)
z=np.zeros(len(bins))
for sp,dr in itertools.product(pL,drugs):
    DFsp=DF[DF.Species==sp]
    DFc=pd.DataFrame({'Year': DFsp['Year'], dr: DFsp[dr]})
    DFc=DFc.dropna()
    if DFc.empty: continue
    Y=get_key(DFc,'Year')
    H=[z for Y in Y]#H will hold the distribution of MIC values each year
    for i1,y in enumerate(Y):
        DFy=DFc[DFc.Year==y]
        h,b1=np.histogram(DFy[dr], bins=bins, density=True)
        H[i1]=h
    H=np.array(H)
    R=np.corrcoef(H)#correlation between the MIC distributions in different years
    if R.size>1:
        np.savetxt('results/correlations/'+sp+'_'+dr+'.txt',R,fmt='%.3f')
        np.savetxt('results/correlations/'+sp+'_'+dr+'_years.txt',Y, fmt='%d')
#COMPUTE TAU FOR EVERY PA PAIR  
from os import path
Dl=[]
for sp,dr in itertools.product(pL,drugs):
    if path.exists('results/correlations/'+sp+'_'+dr+'.txt'):
        R,Y=read_corr(sp, dr)
        n,m=R.shape
        tau,tauN=tautest(R)
        Dl.append([sp,dr,tau,tauN,n])     
Dl=sorted(Dl, reverse=True, key=itemgetter(2))
DFdist=pd.DataFrame.from_records(Dl, columns=['Species', 'Antibiotic', 'Tau', 'TauN', 'N'])
DFdist.to_csv('results/correlations/tau_test.csv', float_format='%.3f', index=False)

# MIC CHANGE
Data files to represent MIC change in clustermaps (Figs 2B, S6A)

In [ ]:
#GLOBAL MIC CHANGE (FOR CLUSTERMAPS)
L=[]
for sp, dr in itertools.product(pL, drugs):
    DFsp=DF[DF.Species==sp]
    M=mic_change(sp,dr,DFsp)
    if not M: continue
    L.append(M)
MIC=pd.DataFrame.from_records(L, columns=['Species', 'Antibiotic', 'MIC change', 'MIC change error', 'Intercept'])
#WRITE TO FILE
MIC.to_csv('data/mic_change_global.csv', na_rep='NaN', float_format='%.3f', index=False)
        
#EUROPEAN MIC CHANGE
RD=pd.read_csv('data/europe_resistance_data.csv')
#GET LIST OF EUROPEAN COUNTRIES FROM ECDC DATA
cR=get_key(RD, 'Country')
for i,c in enumerate(cR):
    if c=='Slovakia':
        cR[i]='Slovak Republic'
DFE=DF[DF.Country.isin(cR)]
for sp,dr in itertools.product(pL, drugs):
    DFsp=DFE[DFE.Species==sp]
    if not M: continue
    M=mic_change(sp,dr,DFsp)
    L.append(M)
MIC=pd.DataFrame.from_records(L, columns=['Species', 'Antibiotic', 'MIC change', 'MIC change error', 'Intercept'])
#WRITE TO FILE
MIC.to_csv('data/mic_change_europe.csv', na_rep='NaN', float_format='%.3f', index=False)

# MIC DISTRIBUTIONS, GAUSSIAN MIXTURE AND R CLUSTER
Datafiles to record the MIC distributions (raw and smooth) of every PA pair for all years (oH and sH), the Gaussian Mixture fit (Z), and the R cluster. Used in Figs 1C, 1D, 3A, 3D, 4C, S7, S8, S12).

In [ ]:
#CREATE NEW DIRECTORY
import os
os.mkdir('results/mic_distributions')
os.mkdir('results/Rcluster')

#GET GAUSSIAN MIXTURE FOR EACH PA PAIR
from functions import *
for sp in pL:
    for dr in drugs:
        if dr not in bP[sp]: continue
        pY,pM,sM=mic_dist(sp,dr,DF,bP[sp][dr][0])
        if not pY: continue
        #GAUSSIAN MIXTURE AND R CLUSTER
        result=False
        while result==False:#sometimes the Gaussian mixture doesn't work
            try:
                extract_rcluster(sp, dr, pY, pM, sM, DF)
                Z,gmm_x=gaussian_mixture(pY, sM)
                result=True
            except:
                pass
        np.savetxt('results/mic_distributions/'+sp+'_'+dr+'.txt',Z,fmt='%.3f')                
        np.savetxt('results/mic_distributions/'+sp+'_'+dr+'_years.txt',pY, fmt='%d')
        #WRITE HISTOGRAMS
        sH=[]
        oH=[]
        for i,y in enumerate(pY):
            # Make regular histogram (SMOOTHMIC)
            B1=np.arange(-10,10.5,0.5)
            n1, bins1=np.histogram(sM[i], bins=B1, density=True)                     
            # Make regular histogram (MIC)
            B2=np.arange(-10,10.5,1)
            n2, bins2=np.histogram(pM[i], bins=B2, density=True)
            sH.append(n1)
            oH.append(n2)
        sH=np.array(sH, dtype=float)
        oH=np.array(oH, dtype=float)
        np.savetxt('results/mic_distributions/'+sp+'_'+dr+'_original.txt',oH,fmt='%.3f')
        np.savetxt('results/mic_distributions/'+sp+'_'+dr+'_smooth.txt',sH,fmt='%.3f')

# CHANGEPOINT ANALYSIS
To check if increased testing has lead to a significant MIC trend in ATLAS (Fig S9). First we need to record global average MIC and number of cases (we had previously computed it for each country):

## GLOBAL AVERAGE MIC PER YEAR 

In [ ]:
L=[]
for sp in pL:#for every pathogen
    DFsp=DF[DF.Species==sp]
    if DFsp.empty: continue
    for dr in drugs:#for every drug
        DFnew=pd.DataFrame({'Year': DFsp['Year'], dr: DFsp[dr]})
        DFnew=DFnew.dropna()
        if DFnew.empty: continue
        yL=get_key(DFnew, 'Year')
        for y in yL:#for every year
            DFy=DFnew[DFnew.Year==y]
            if DFy.empty: continue
            #DATA
            BP=0#we include those PA pairs without a breakpoint
            if dr in bP[sp]:
                BP=bP[sp][dr][0]
            m=DFy[dr].mean()-BP
            merr=DFy[dr].sem()
            N=len(DFy)
            #RECORD
            L.append([sp,dr,y,m,merr,N])
#SAVE ALL IN DATAFRAME AND WRITE TO FILE
MICdf=pd.DataFrame.from_records(L, columns=['Species', 'Antibiotic', 'Year', 'Average MIC', 'Average MIC SEM', 'Number of cases'])
MICdf.to_csv('data/average_mic_per_year_complete.csv', float_format='%.3f', na_rep='NaN', index=False)

## CHANGEPOINT DETECTION

In [ ]:
DF, pL, bP, drugs=read_dataset('data/',complete=True)
DF2=pd.read_csv('data/average_mic_per_year_complete.csv')
sdL=[]
for sp, dr in itertools.product(pL, drugs):
    DFsp=DF2[(DF2.Species==sp) & (DF2.Antibiotic==dr)]
    if DFsp.empty: continue
    yL=get_key(DFsp, 'Year')
    count+=1
    if len(yL)<2: continue
    nL=[]
    rL=[]
    for year in yL:
        DFnew=DFsp[DFsp.Year==year]
        n=DFnew['Number of cases'].sum()
        nL.append(n)
    tau=find_changepoint(nL)
    if tau>-1:
        sdL.append([sp,dr, yL[tau]])

#NOW CHECK WHICH OF THOSE HAVE A NEGATIVE SLOPE
special=[]
from scipy import stats
for key in sdL:
    sp=key[0]
    dr=key[1]
    BP=0
    if dr in bP[sp]:
        BP=bP[sp][dr][0]
    tau=key[2]
    #COMPUTE SLOPE
    DFdr=DF[DF.Species==sp]
    DFc=pd.DataFrame({'Year': DFdr['Year'], dr: DFdr[dr]})
    DFc=DFc.dropna()
    slope, intercept, r_value, p_value, std_err=stats.linregress(DFc['Year'], DFc[dr])
    intercept=intercept-BP
    if p_value>0.05:
        slope=0
        intercept=DFc[dr].mean()-BP
    #COMPUTE SECOND SLOPE
    DFdr=DF[(DF.Species==sp) & (DF.Year>tau-1)]
    DFc=pd.DataFrame({'Year': DFdr['Year'], dr: DFdr[dr]})
    DFc=DFc.dropna()
    yL=sorted(get_key(DFc, 'Year'))
    s1=0
    i1=DFc[dr].mean()-BP
    if len(yL)>2:
        s1, i1, r1, p1, std1=stats.linregress(DFc['Year'], DFc[dr])
        i1=i1-BP
        if p1>0.05:
            s1=0
    #COMPUTE THIRD SLOPE
    DFdr=DF[(DF.Species==sp) & (DF.Year<tau)]
    DFc=pd.DataFrame({'Year': DFdr['Year'], dr: DFdr[dr]})
    DFc=DFc.dropna()
    yL=sorted(get_key(DFc, 'Year'))
    s2=0
    i2=DFc[dr].mean()-BP
    if len(yL)>2:
        s2, i2, r2, p2, std2=stats.linregress(DFc['Year'], DFc[dr])
        i2=i2-BP
        if p2>0.05:
            s2=0
    if slope<0:
        special.append([sp,dr,tau,s1,i1,s2,i2,slope,intercept])

DFstr=pd.DataFrame.from_records(special, columns=['Species', 'Antibiotic', 'Changepoint', 'Slope After', 'Intercept After', 'Slope Before', 'Intercept Before', 'Slope Global', 'Intercept Global'])
DFstr.to_csv('data/changepoint_analysis.csv', index=False, float_format='%.3f')

# R CLUSTER TRENDS

Data for Figs 3A, 3B, 3C, 3D, 4A, 4B, 4C, S6B, S10.

## GLOBAL TREND
(record slopes and intercepts with %.6f, so that there is no error!)

In [ ]:
DF, pL, bP, drugs=read_dataset('data/')
from sklearn import mixture
import statsmodels.api as sm
from functions import *

lS=[]
lA=[]
lT=[]
lI=[]
lEr=[]
lRt=[]
lRte=[]
lSt=[]
lSte=[]
lRv=[]
lRve=[]
lRI=[]
lSv=[]
lSve=[]
lSI=[]
lB=[]
lE=[]
lE2=[]
N=50
for sp in pL:
    for dr in drugs:
        print(sp,dr)
        BP=0
        BPbool=False
        if dr in bP[sp]:
            BP=bP[sp][dr][0]
            BPbool=True
        #GLOBAL TREND
        DFsp=DF[DF.Species==sp]
        DFc=pd.DataFrame({'Year': DFsp['Year'], 'MIC': DFsp[dr]})
        DFc=DFc.dropna()
        yL=get_key(DFc, 'Year')
        if len(yL)<2: continue
        x1=DFc['Year'].tolist()
        y1=DFc['MIC'].tolist()
        s1,i1,r1,p1,std1=stats.linregress(x1,y1)
        if p1<0.05:
            T=s1
            Er=std1
            I=i1
        else:
            T=0
            Er=0
            I=DFc['MIC'].mean()
        #LOOP FOR R CLUSTER
        Rtrend=[]
        Rvalue=[]
        Ryear=[]
        R2017=[]
        Strend=[]
        Svalue=[]
        Syear=[]
        S2017=[]
        i=-1
        while i<N:
            i=i+1
            #GET R CLUSTER
            try:#sometimes the Gaussian mixture does not converge, we need to generate a new smooth distribution
                pY,pM,sM=mic_dist(sp,dr,DF,0)
                mDFc, lDFc=extract_rclusternowrite(sp, dr, pY, pM, sM, DF)
            except:
                i=i-1
                continue
            #RCLUSTER TREND
            yLm=get_key(mDFc, 'Year') 
            if len(yLm)>=2:
                x1=mDFc['Year'].tolist()
                y1=mDFc['MIC'].tolist()
                s1,i1,r1,p1,std1=stats.linregress(x1,y1)
                if p1<0.05:
                    Rtrend.append(s1)#Rs=s1
                    mDFy=mDFc[mDFc.Year==2017]
                    if mDFy.empty:
                        R2017.append(s1*2017+i1)
                        E=True
                    else:
                        R2017.append(mDFy['MIC'].mean())
                        E=False
                else:
                    Rtrend.append(0.0)#Rs=0
                    R2017.append(mDFc['MIC'].mean())
                Rvalue.append(np.average(y1))
                Ryear.append(np.average(x1))
            #SCLUSTER TREND
            yLm=get_key(lDFc, 'Year') 
            if len(yLm)>=2:
                x1=lDFc['Year'].tolist()
                y1=lDFc['MIC'].tolist()
                s1,i1,r1,p1,std1=stats.linregress(x1,y1)
                if p1<0.05:
                    Strend.append(s1)#Rs=s1
                    lDFy=lDFc[lDFc.Year==2017]
                    if lDFy.empty:
                        S2017.append(s1*2017+i1)
                        E2=True
                    else:
                        S2017.append(lDFy['MIC'].mean())
                        E2=False
                else:
                    Strend.append(0.0)#Rs=0
                    S2017.append(lDFc['MIC'].mean())
                Svalue.append(np.mean(y1))
                Syear.append(np.mean(x1))
        #LISTS
        lS.append(sp)
        lA.append(dr)
        lT.append(T)
        lEr.append(Er)
        lI.append(I-BP)
        lRt.append(np.mean(Rtrend))
        lRte.append(np.std(Rtrend))
        lRv.append(np.mean(R2017)-BP)
        lRve.append(np.std(R2017))
        lRI.append(np.mean(Rvalue)-BP-np.mean(Rtrend)*np.mean(Ryear))
        lSt.append(np.mean(Strend))
        lSte.append(np.std(Strend))
        lSv.append(np.mean(S2017)-BP)
        lSve.append(np.std(S2017))
        lSI.append(np.mean(Svalue)-BP-np.mean(Strend)*np.mean(Syear))
        lB.append(BPbool)
        lE.append(E)
        lE2.append(E2)
        newDF=pd.DataFrame({'Species': lS, 'Antibiotic': lA, 'Trend':lT,
                    'Trenderror':lEr, 'Intercept': lI,
                    'Rtrend':lRt, 'Rtrenderror': lRte, 
                    'R2017': lRv, 'R2017error': lRve,
                    'RIntercept': lRI,
                    'Strend': lSt, 'Strenderror': lSte,
                    'S2017': lSv, 'S2017error': lSve,
                    'SIntercept': lSI, 'Breakpoint': lB,
                    'EstimatedR': lE, 'EstimatedS': lE2})
        newDF.to_csv('results/Rcluster/resistant_cluster_trends_noBP.csv', index=False, float_format='%.6f')

## EUROPE TREND

In [ ]:
RD=pd.read_csv('data/europe_resistance_data.csv')
#GET LIST OF EUROPEAN COUNTRIES FROM ECDC DATA
cR=get_key(RD, 'Country')
for i,c in enumerate(cR):
    if c=='Slovakia':
        cR[i]='Slovak Republic'

DF, pL, bP, drugs=read_dataset('data/')
DF=DF[DF.Country.isin(cR)]
from sklearn import mixture
import statsmodels.api as sm

lS=[]
lA=[]
lT=[]
lEr=[]
lRt=[]
lRte=[]
N=50
for sp in pL:
    for dr in drugs:
        print(sp,dr)
        BP=0
        if dr in bP[sp]:
            BP=bP[sp][dr][0]
        #GLOBAL TREND
        DFsp=DF[DF.Species==sp]
        DFc=pd.DataFrame({'Year': DFsp['Year'], 'MIC': DFsp[dr]})
        DFc=DFc.dropna()
        yL=get_key(DFc, 'Year')
        if len(yL)<2: continue
        x1=DFc['Year'].tolist()
        y1=DFc['MIC'].tolist()
        s1,i1,r1,p1,std1=stats.linregress(x1,y1)
        if p1<0.05:
            T=s1
            Er=std1
            I=i1
        else:
            T=0
            Er=0
            I=DFc['MIC'].mean()
        #LOOP FOR R CLUSTER
        Rtrend=[]
        Rvalue=[]
        Strend=[]
        Svalue=[]
        i=-1
        while i<N:
            i=i+1
        # for i in range(N):
            print(i,sp,dr)
            #GET R CLUSTER
            try:
                pY,pM,sM=mic_dist(sp,dr,DF,0)
                mDFc, lDFc=extract_rclusternowrite(sp, dr, pY, pM, sM, DF)
                mDFc=mDFc[mDFc.Country.isin(cR)]
            except:
                print('Problem encountered when extracting R cluster!',i)
                i=i-1
                # print(i)
                continue
            #RCLUSTER TREND
            yLm=get_key(mDFc, 'Year') 
            if len(yLm)>=2:
                x1=mDFc['Year'].tolist()
                y1=mDFc['MIC'].tolist()
                s1,i1,r1,p1,std1=stats.linregress(x1,y1)
                if p1<0.05:
                    Rtrend.append(s1)#Rs=s1
                else:
                    Rtrend.append(0.0)#Rs=0
        Rtrend=np.array(Rtrend)
        #LISTS
        lS.append(sp)
        lA.append(dr)
        lT.append(T)
        lEr.append(Er)
        lRt.append(np.mean(Rtrend))
        lRte.append(np.std(Rtrend))

        newDF=pd.DataFrame({'Species': lS, 'Antibiotic': lA, 'Trend':lT,
                    'Trenderror':lEr, 
                    'Rtrend': lRt, 'Rtrenderror': lRte})
        newDF.to_csv('results/Rcluster/resistant_cluster_trends_noBP_europe.csv', index=False, float_format='%.3f')